In [ ]:
from nltk import PCFG,InsideChartParser,ViterbiParser

In [ ]:
grammar =  PCFG.fromstring("""
S -> NP VP [1.0]
VP -> V NP [0.7] | VP PP [0.3]
NP -> Det N [0.5] | NP PP [0.2] | 'John' [0.3]
PP -> P NP [1.0]
V -> 'saw' [0.2] | 'ate' [0.8]
Det -> 'the' [0.6] | 'a' [0.4]
N -> 'man' [0.5] | 'telescope' [0.5]
P -> 'with' [0.4] | 'in' [0.6]
""")
parser=InsideChartParser(grammar)
sentence = "John saw a man with a telescope"
tokens=sentence.split()
count=1
for tree in parser.parse(tokens):
    print("TREE",count,end="\n\n\n")
    tree.pretty_print()
    print(tree,end="\n\n\n")
    count+=1
    

In [ ]:
parser=ViterbiParser(grammar)
for tree in parser.parse(tokens):
    print(tree)
    tree.pretty_print()

In [ ]:
from nltk import PCFG
from nltk import word_tokenize
pcfg=PCFG.fromstring("""
S -> NP VP [1.0]
PP -> P NP [1.0]
VP -> V NP [0.7]
VP -> VP PP [0.3]
P -> 'with' [1.0]
V ->'saw' [1.0]
NP -> NP PP [0.4]
NP -> 'Mani' [0.1]
NP -> 'ears' [0.18]
NP -> 'saw' [0.04]
NP -> 'stars' [0.18]
NP -> 'telescope' [0.1]
"""
)
text="Mani saw stars with ears"
tokens=word_tokenize(text)
n=len(tokens)
print(tokens)
table=[[[]for i in range(n)]for i in range(n)]
print(table)
for i in range(n):
    for prod in pcfg.productions(rhs=tokens[i]):
        if table[i][i] is None:
            table[i][i]=[(prod.lhs(),prod.prob())]
        else:
            table[i][i].append([prod.lhs(),prod.prob()])
print(table)            
for length in range(2,n+1):
    for i in range(n-length+1):
        j=i+length-1
        for k in range(i,j):
            for prod in pcfg.productions():
                for left,leftprob in table[i][k]:
                    for right,rightprob in table[k+1][j]:
                        if prod.rhs()==(left,right):
                            prob=leftprob*rightprob*prod.prob()
                            if table[i][j] is None:
                                table[i][j]=[(prod.lhs(),prob)]
                            else:
                                table[i][j].append([prod.lhs(),prob])
print(table)                                
if table[0][n-1] is not None:
    if pcfg.start() in (x for (x,prob) in table[0][n-1]):
        print("Sentence is valid.")
        for (x,prob) in table[0][n-1]:
            print("Probability of sentence is ",prob)   